In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.
  return image, label

In [4]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(num_examples)
print(num_classes)

23262
2


In [11]:
module_selection = ('mobilenet_v2', 224, 1280)
handle_base, pixels, FV_SIZE = module_selection

MODULE_HANDLE = 'https://tfhub.dev/google/tf2-preview/{}/feature_vector/4'.format(handle_base)
IMAGE_SIZE = (pixels, pixels)

In [12]:
BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_validation.map(format_image).batch(1)

In [13]:
feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=IMAGE_SIZE + (3,),
    output_shape=[FV_SIZE],
    trainable=False
)

model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
hist = model.fit(
    train_batches,
    epochs=1,
    validation_data=validation_batches
)

582/582 [==============================] - 680s 1s/step - loss: 0.0406 - accuracy: 0.9867 - val_loss: 0.0371 - val_accuracy: 0.9880


In [17]:
CATS_VS_DOGS_SAVED_MODEL = 'exp_saved_model'
tf.saved_model.save(model, CATS_VS_DOGS_SAVED_MODEL)

## Convert to tflite

In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

## Use the model

In [19]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

In [22]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(interpreter.get_input_details())
print(interpreter.get_output_details())

[{'name': 'serving_default_keras_layer_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 177, 'shape': array([1, 2], dtype=int32), 'shape_signature': array([-1,  2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [26]:
predictions = []

test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

In [27]:
score = 0
for item in range(100):
  prediction = np.argmax(predictions[item])
  label = test_labels[item]
  if prediction == label:
    score += 1

print('Out of 100 predictions I got: {} correct'.format(score))

Out of 100 predictions I got: 97 correct


## Optimize the model

In [29]:
converter = tf.lite.TFLiteConverter.from_saved_model(CATS_VS_DOGS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

In [30]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

predictions = []

test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

score = 0
for item in range(100):
  prediction = np.argmax(predictions[item])
  label = test_labels[item]
  if prediction == label:
    score += 1

print('Out of 100 predictions I got: {} correct'.format(score))

Out of 100 predictions I got: 97 correct
